# MNIST + CNN + Keras

>[MNIST + CNN + Keras](#scrollTo=ZEVoy5LcVzXu)

>>>[Define the model](#scrollTo=lPZ8UGKPVycX)

>>>[Prepare the dataset](#scrollTo=b2TsLHHXWIvc)

>>>[Train and score:](#scrollTo=6gsJ9sHtWbQC)

>[To show the tensorboard](#scrollTo=cyp_Ga9UmRiQ)



In [0]:
from keras import backend as K
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.models import Model
from keras.datasets import mnist
import keras



---
### Define the model

In [0]:
def cnn_model_fn():

    #input shape = [size, 28, 28, 1]
    input_layer = Input(shape=[28, 28, 1], name='input')

    # Convolutional Layer #1 . out size*32*28*28
    conv1 = Conv2D(filters=32, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)(input_layer)
        # tf.layers.conv2d(inputs=input_layer, filters=32, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
    # Pooling Layer #1 . out size*32*14*14
    pool1 = MaxPooling2D(pool_size=[2, 2], strides=2)(conv1)
        # tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2 . out size*64*14*14
    conv2 = Conv2D(filters=64, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)(pool1)
        # tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
    # Pooling Layer #2 . out size*64*7*7
    pool2 = MaxPooling2D(pool_size=[2, 2], strides=2)(conv2)
        # tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Dense Layer
    pool2_flat = Flatten()(pool2)
        # tf.reshape(pool2, [-1, 7 * 7 * 64])
    dense = Dense(units=1024, activation=tf.nn.relu)(pool2_flat)
        # tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # dropout
    dropout = Dropout(rate=0.4)(dense)
        # tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = Dense(units=10, activation='softmax')(dropout)
        # tf.layers.dense(inputs=dropout, units=10)
    model = Model(inputs=input_layer, outputs=logits)
    model.summary()
    
    return model
  

---
###Prepare the dataset 

from **keras.datasets.mnist**





In [2]:
# prepare data
# mnist_images = input_data.read_data_sets("MNIST_data/", one_hot=True)

#shape of x_train is [60000, 28, 28]
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
    x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)
    input_shape = (1, 28, 28)
else:
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)



x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


---
###Train and score
1.   Compile the model
2.   Fit the data (train)
3.   Get score (accuracy) between 0.0~1.0



In [35]:
model = cnn_model_fn()

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
tbcallback = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

model.fit(x_train, y_train,
          batch_size=64,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[tbcallback])
score = model.evaluate(x_test, y_test, verbose=0)
model.save_weights('mnist_cnn.h5')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              3212288   
__________

60000/60000 [==============================] - 16s 260us/step - loss: 0.1179 - acc: 0.9632 - val_loss: 0.0331 - val_acc: 0.9897
Test loss: 0.033088591791741784
Test accuracy: 0.9897


# To show the tensorboard

In [1]:
!ls
!pwd

datalab
/content


In [0]:
LOG_DIR = '/content/logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [3]:
! curl http://localhost:6006

curl: (7) Failed to connect to localhost port 6006: Connection refused


In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1
! unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [6]:
#@title Default title text
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://e85faccd.ngrok.io


# Save file to google drive

Get python lib from github

In [29]:
!git clone https://github.com/kk60258/colab_utils.git


Cloning into 'colab_utils'...
remote: Counting objects: 216, done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 216 (delta 15), reused 40 (delta 12), pack-reused 171
Receiving objects: 100% (216/216), 60.19 KiB | 7.52 MiB/s, done.
Resolving deltas: 100% (83/83), done.


###Upload to google cloud bucket gs://ai-model-test-ml/

In [39]:
import os
import colab_utils.gcloud

# authorize access to Google Cloud SDK from `colaboratory` VM
project_name = "ai-model-test"
bucket_name = "ai-model-test-ml"

colab_utils.gcloud.gcloud_auth(project_name)
# colab_utils.gcloud.config_project(project_name)

tar_filepath = "/content/mnist_cnn.h5"
bucket_path = "gs://{}/{}".format(bucket_name, "mnist_cnn.h5")
project_id = project_name
colab_utils.gcloud.gcs_upload(tar_filepath, bucket_path, project_id=project_id)

# set paths
#ROOT = %pwd
#LOG_DIR = os.path.join(ROOT, 'log')

#TRAIN_LOG = os.path.join(LOG_DIR, 'training-run-1')

# save latest checkpoint as a zipfile to a GCS bucket `gs://my-checkpoints/`
#     zipfile name = "{}.{}.zip".format() os.path.basename(TRAIN_LOG), global_step)
#                     e.g. gs://my-checkpoints/training-run-1.1000.zip"
#bucket_name = "my-checkpoints"
#colab_utils.gcloud.save_to_bucket(TRAIN_LOG, bucket_name, project_name, save_events=True, force=False)


"""
# restore a zipfile from GCS bucket to a local directory, usually in  
#     tensorboard `log_dir`
CHECKPOINTS = os.path.join(LOG_DIR, 'training-run-2')
zipfile = os.path.basename(TRAIN_LOG)   # training-run-1
colab_utils.gcloud.load_from_bucket("training-run-1.1000.zip", bucket_name, CHECKPOINTS )


# mount gcs bucket to local fs using the `gcsfuse` package, installs automatically
bucket = "my-bucket"
local_path = colab_utils.gcloud.gcsfuse(bucket=bucket)  
# gcsfuse(): Using mount point: /tmp/gcs-bucket/my-bucket

!ls -l local_path
!umount local_path
"""

uploading file=gs://ai-model-test-ml/mnist_cnn.h5 ...


'\n# restore a zipfile from GCS bucket to a local directory, usually in  \n#     tensorboard `log_dir`\nCHECKPOINTS = os.path.join(LOG_DIR, \'training-run-2\')\nzipfile = os.path.basename(TRAIN_LOG)   # training-run-1\ncolab_utils.gcloud.load_from_bucket("training-run-1.1000.zip", bucket_name, CHECKPOINTS )\n\n\n# mount gcs bucket to local fs using the `gcsfuse` package, installs automatically\nbucket = "my-bucket"\nlocal_path = colab_utils.gcloud.gcsfuse(bucket=bucket)  \n# gcsfuse(): Using mount point: /tmp/gcs-bucket/my-bucket\n\n!ls -l local_path\n!umount local_path\n'